# GPU Environment Tensorflow

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/gpu-environment-tensorflow](https://github.com/huseinzol05/Malaya/tree/master/example/gpu-environment-tensorflow).
    
</div>

In [1]:
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
%%time

import malaya
import logging
logging.basicConfig(level = logging.INFO)

CPU times: user 3.24 s, sys: 3.44 s, total: 6.69 s
Wall time: 2.65 s


/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3372
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3890
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


### List available GPU

**You must install Tensorflow GPU version first to enable GPU hardware acceleration**.

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

2022-10-22 22:50:22.823675: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5915678662307780302,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 18086494208
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13854386303779722161
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"]

2022-10-22 22:50:22.827680: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:22.832081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:22.832743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:23.365376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:23.366233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

### Run model inside GPU

We can follow steps from here https://www.tensorflow.org/guide/gpu

In [4]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

In [5]:
malaya.sentiment.available_transformer()

INFO:malaya.sentiment:tested on test set at https://github.com/huseinzol05/malay-dataset/tree/master/sentiment/semisupervised-twitter-3class


,Size (MB),Quantized Size (MB),macro precision,macro recall,macro f1-score
bert,425.6,111.00,0.93182,0.93442,0.93307
tiny-bert,57.4,15.40,0.93390,0.93141,0.93262
albert,48.6,12.80,0.91228,0.91929,0.91540
tiny-albert,22.4,5.98,0.91442,0.91646,0.91521
xlnet,446.6,118.00,0.92390,0.92629,0.92444
alxlnet,46.8,13.30,0.91896,0.92589,0.92198


In [6]:
with tf.device('/device:GPU:0'):
    tiny_albert = malaya.sentiment.transformer(model = 'tiny-albert')

2022-10-22 22:50:23.644489: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:23.645216: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:23.645807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:23.646643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:50:23.647257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [7]:
tiny_albert.predict_proba(['hello'])

2022-10-22 22:50:27.931657: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[{'negative': 0.001145982, 'neutral': 0.99809974, 'positive': 0.0007543671}]